In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizerFast
import matplotlib.pyplot as plt

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def analyze_sentiment(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return pred

df = pd.read_csv('cafe_reviews-1.csv')

df['predicted_sentiment'] = df['text'].apply(analyze_sentiment)

labels_map = {0: "Neutral", 1: "Positive", 2: "Negative"}
df['sentiment_label'] = df['predicted_sentiment'].map(labels_map)

counts = df['sentiment_label'].value_counts().reindex(['Positive', 'Neutral', 'Negative'], fill_value=0)
counts.plot(kind='bar', color=['green', 'gray', 'red'])
plt.xlabel("Тональность")
plt.ylabel("Количество отзывов")
plt.title("Распределение тональности отзывов (Predicted)")
plt.show()
